In [9]:
import os

import numpy as np
import pandas as pd
# Torch
import torch
import torchvision
from torchvision import datasets,transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib
from torchvision import models
from sklearn.metrics import precision_score, f1_score

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
transformer = transforms.Compose([
    # Resize the image to (160, 160)
    transforms.Resize((160, 160)),
    # Convert the image to grayscale only one channel
    transforms.Grayscale(num_output_channels=1),

    #Data Augmentation
    transforms.RandomHorizontalFlip(),       # Randomly flip the image horizontally
    transforms.RandomRotation(degrees=15),   # Randomly rotate the image by up to 15 degrees
    transforms.ToTensor(),                   # Convert the image to a PyTorch tensor
])


In [5]:

test_path = '/content/drive/MyDrive/Colab Notebooks/DL Submission/test'
test_loader = DataLoader(torchvision.datasets.ImageFolder(
                          test_path,
                          transform=transformer),
                          batch_size = 128,
                          shuffle=False)

In [6]:
class ConvNet(nn.Module):
  def __init__(self,num_classes=2):
    super(ConvNet,self).__init__()
    # input shape (256,1, 160,160)

    #output shape ((width-kernel+2P)/stride)+1 160-3+2+1=160
    self.conv1 =nn.Conv2d(in_channels=1,out_channels=20,kernel_size=3,stride=1,padding=1) # outptu channels is the number of kernels used
    self.bn1 = nn.BatchNorm2d(num_features=20)
    self.sigmoid1 = nn.Sigmoid()

    self.conv2 = nn.Conv2d(in_channels=20,out_channels = 32,kernel_size =3,stride=1,padding=1)
    #256*20*80*80
    self.bn2=nn.BatchNorm2d(num_features=32)
    self.pool=nn.MaxPool2d(kernel_size=2)
    #shape = 256*12*80*80 (reduce image size by 2)
    self.sigmoid2 = nn.Sigmoid()

    #256*20*80*80
    self.conv3 = nn.Conv2d(in_channels=32,out_channels =64,kernel_size =3,stride =1 ,padding=1)
#     #256*32*80*80
    self.bn3 = nn.BatchNorm2d(num_features=64)
    self.sigmoid3 =nn.Sigmoid()

    self.fc = nn.Linear(in_features=64*80*80,out_features = 2)

  def forward(self,input):
    output = self.conv1(input)
    output = self.bn1(output)
    output = self.sigmoid1(output)
    output = self.conv2(output)
    output = self.bn2(output)
    output = self.pool(output)
    output = self.sigmoid2(output)
    output = self.conv3(output)
    output = self.bn3(output)
    output = self.sigmoid3(output)

    output = output.view(-1,64*80*80)
    output = self.fc(output)
    return output


In [7]:
model = ConvNet(num_classes=2).to(device)

Testing on test data(unseen data)

In [10]:
model_path = '/content/drive/MyDrive/Colab Notebooks/DL Submission/Best_model_CNN1_Sig_adagrad_128_0.001.pt'
state_dict = torch.load(model_path,map_location=device)

model.load_state_dict(state_dict)
model.to(device)

#to calulate eval later
predicted_list = []
target_list =[]
#set to evaluation mode
model.eval()

total_correct = 0
total_images_test = len(test_loader.dataset)
with torch.no_grad():
    for inputs, targets in test_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_correct += (predicted == targets).sum().item()
        predicted_list.extend(predicted.cpu().numpy())
        target_list.extend(targets.cpu().numpy())

accuracy = total_correct / total_images_test
print(f'Test Accuracy: {accuracy * 100:.2f}%')

precision = precision_score(target_list, predicted_list, average='macro')
print(f'Precision: {precision * 100:.2f}%')

f1 = f1_score(target_list, predicted_list, average='macro')
print(f'F1 Score: {f1 * 100:.2f}%')


Test Accuracy: 85.42%
Precision: 86.22%
F1 Score: 83.69%


Testing Single Images

In [ ]:
import torch
from PIL import Image


# Load the image
image_path = '/content/drive/MyDrive/Colab Notebooks/good_lungs.png'
image = Image.open(image_path)

# Apply transformations
input_image = transformer(image).unsqueeze(0)  # Add a batch dimension

# Load the model
model = ConvNet(num_classes=2)
model_path = '/content/drive/MyDrive/Colab Notebooks/DL Submission/Best_model_CNN1_Sig_adagrad_128_0.001.pt'
state_dict = torch.load(model_path,map_location=device)
model.load_state_dict(state_dict)
model.eval()

# Make prediction
with torch.no_grad():
    output = model(input_image)
    _, predicted_class = torch.max(output, 1)

# Print prediction
if predicted_class==0:
  print('NORMAL')
if predicted_class==1:
  print('PNEMONIA')

NORMAL
